# Consolidated File - Auction.py & Bidder.py

In [8]:
from numpy import random
from numpy.random import randint


class User:

    def __init__(self):
        self.__probability = random.uniform(0, 1)

    def get_probability(self):
        return self.__probability

    def show_ad(self):

        # draw a random number to simulate whether the user clicked
        click_threshold = random.uniform(0, 1)

        # compare the random draw to the secret probability to represent if the user clicked
        if click_threshold <= self.__probability:
            return True
        return False

class Auction:

    def __init__(self, users, bidders):

        self.users = users
        self.bidders = bidders
        # initialize all balances to 0
        self.balances = {bidder: 0 for bidder in bidders}

    def execute_round(self):

        # select a random user for the round
        random_user = randint(0, len(self.users))

        bids = {}
        for bidder in self.bidders:
            # call each bidder's bid method to place the bids
            bids[bidder] = bidder.bid(random_user)

        # select the winning bidder and winning price (second highest bid)
        highest_bid = 0
        winning_price = 0
        for bidder, bid_value in bids.items():
            # evaluate the bid to determine if it is the highest
            # the highest bidder wins the round
            if bid_value > highest_bid:
                # set the winning bid price to the previous high bid
                winning_price = highest_bid
                highest_bid = bid_value
            # this case is for when the bid price < highest bid but > the current winning price
            elif bid_value > winning_price:
                winning_price = bid_value

        # tie-breaking logic
        winning_bidders = []
        # create a list of all bidders who bid the highest price
        for bidder in self.bidders:
            if bids[bidder] == highest_bid:
                winning_bidders.append(bidder)
        # randomly select the winning bidder from this list
        winning_bidder_index = (
            0 if len(winning_bidders) == 1 else randint(0, len(winning_bidders))
        )
        winning_bidder = winning_bidders[winning_bidder_index]

        # show the ad to the user
        ad_result = self.users[random_user].show_ad()

        for bidder in self.bidders:
            if bidder == winning_bidder:
                # send notification to the winning bidder and update the balance
                bidder.notify(True, winning_price, ad_result)
                self.balances[bidder] -= winning_price
                if ad_result:
                    self.balances[bidder] += 1
            else:
                # send notification to the losing bidders
                bidder.notify(False, winning_price, None)

class Bidder:

    def __init__(self, num_users, num_rounds):

        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0

        # clearing prices for each user
        self.user_clearing_prices = {i: [] for i in range(num_users)}

        # dictionary mapping users to number of bids won for a user
        self.user_wins = {i: 0 for i in range(num_users)}
        
        # dictionary mapping the number of clicks for a user
        self.user_clicks = {i: 0 for i in range(num_users)}
        
        # the ratio of clicks to winning bids
        self.user_click_ratio = {i: 0. for i in range(num_users)}
        
        # current round number
        self.current_round_user_id = -1

        # percentage of users to explore before starting exploitation
        self.exploration_users = .25
        
        # if there are a small number of users relative to the number
        # of rounds, explore all users
        if self.num_users / self.num_rounds <= .2:
            self.exploration_users = 1       
        
        # bid price during exploration
        self.exploration_bid_price = 1.
        self.default_bid_price = 0.5

        # next bid for a specific user
        # self.next_bid = {user_id : self.unseen_user_bid_price for user_id in range(num_users)}
        self.next_bid = {i: self.default_bid_price for i in range(num_users)}

        # set max bid amounts for high/med/low confidence bids
        self.high_bid_max = 0.99
        self.med_bid_max = 0.65
        self.low_bid_max = 0.1

        # adders applied to bids based on the last clearing price
        self.large_adder = 0.01
        self.small_adder = 0.005

        # phase of algorithm - explore or exploit
        self.phase = 'explore'

    def bid(self, user_id):

        self.bidding_round += 1
        self.current_round_user_id = user_id
        bid_price = self.default_bid_price        

        print(f"Phase {self.phase}")

        # exploit if in exploitation phase, or the user has been seen before
        if self.user_wins[self.current_round_user_id] > 0 or self.phase == 'exploit':
            bid_price = self.next_bid[self.current_round_user_id]
        else:
            # if in exploration phase, try to win the round
            bid_price = self.exploration_bid_price
        
        print(f"Smart {id(self)} bid {bid_price} for user {user_id}")
        return bid_price

    def notify(self, auction_winner, price, clicked):

        # dynamically adjust the exploration bid based on the last bid price
        # self.exploration_price = max(self.exploration_price, min(price +
        # (price * self.small_adder), self.high_bid_max))

        # update the list of clearing prices for this user
        self.user_clearing_prices[self.current_round_user_id].append(price)
        print(f"Winning bid price  was {price}")

        if auction_winner:
            # update the list of total winning bids for this user
            self.user_wins[self.current_round_user_id] += 1

            if clicked:
                # update the number of clicks for this user
                self.user_clicks[self.current_round_user_id] += 1
                print(
                    f"Smart {id(self)} won round; User {self.current_round_user_id}"
                    f" clicked"
                )
            else:
                print(
                    f"Smart {id(self)} Won round; User {self.current_round_user_id}"
                    f" did not click"
                )
                pass
                
            # calculate the click ratio for the user
            if self.user_wins[self.current_round_user_id] > 0:                    
                self.user_click_ratio[self.current_round_user_id] = self.user_clicks[self.current_round_user_id] / self.user_wins[self.current_round_user_id]
            else:
                self.user_click_ratio[self.current_round_user_id] = 0
            

            # # calculate the next bid for this user based on history
            # if self.user_click_ratio.get(self.current_round_user_id, 0) > 0.8:
            #     # if the user is likely to click
            #     # use the last clearing price for this user plus a large adder
            #     self.next_bid[self.current_round_user_id] = min(
            #         self.user_clearing_prices[self.current_round_user_id][-1]
            #         + (
            #             self.user_clearing_prices[self.current_round_user_id][-1]
            #             * self.large_adder
            #         ),
            #         self.high_bid_max,
            #     )
            # elif self.user_click_ratio.get(self.current_round_user_id, 0) > 0.5:
            #     # if the user is somewhat likely to click
            #     # use the last clearing price for this user plus a small adder
            #     self.next_bid[self.current_round_user_id] = min(
            #         self.user_clearing_prices[self.current_round_user_id][-1]
            #         + (
            #             self.user_clearing_prices[self.current_round_user_id][-1]
            #             * self.small_adder
            #         ),
            #         self.med_bid_max,
            #     )
            # else:                    
            #     # if the user is not likely to click
            #     # use a static value
            #     self.next_bid[self.current_round_user_id] = min(
            #         self.user_clearing_prices[self.current_round_user_id][-1]
            #         - (
            #             self.user_clearing_prices[self.current_round_user_id][-1]
            #             * self.large_adder
            #         ),
            #         self.high_bid_max,
            #     )

        # determine the phase based on the number of users who've been explored
        if len(self.user_wins.keys()) / self.num_users >= self.exploration_users or len(self.user_wins.keys()) == self.num_users:
            self.phase = 'exploit'
        
        print(f'Next bid for user {self.current_round_user_id} is {self.next_bid[self.current_round_user_id]}')

In [9]:
c1, c2, c3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)

In [10]:
#auction = Auction( [User() for i in range(100)], [c1, c2, c3])
auction = Auction( [User()], [c1, c2, c3])

In [11]:
auction.execute_round()

Phase explore
Smart 140211232922640 bid 1.0 for user 0
Phase explore
Smart 140211232919952 bid 1.0 for user 0
Phase explore
Smart 140211232921008 bid 1.0 for user 0
Winning bid price  was 1.0
Next bid for user 0 is 0.5
Winning bid price  was 1.0
Smart 140211232919952 Won round; User 0 did not click
Next bid for user 0 is 0.5
Winning bid price  was 1.0
Next bid for user 0 is 0.5
